In [1]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os

from web3 import Web3, middleware, Account
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from pprint import pprint

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

# Import constants.py and necessary functions from bit and web3
from constants import *

# Connect Web3
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

# Enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
 
# Create a function called `derive_wallets`
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)


# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether"),
        gasEstimate = w3.eth.estimateGas({ "to": to, "from": account, "amount": value }),
        return {
            "to": to,
            "from": account,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account),
            "chainId": w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account.address, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

In [2]:
coins['btc-test']

[{'path': "m/44'/1'/0'/0/0",
  'address': 'mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD',
  'xprv': 'tprv8khhNkzGYHzyhzgCwAKEf4xVvkBbV2vWqBQpybEMhGHivv3WgwhnjdCguDwQ1ujRnN5JWEM8q8nE8zjHSrthSTvMcxb6V575s5x2zT3q1Pk',
  'xpub': 'tpubDHPjXB2WgfgebThzpoyq4UccVmhXeN7RQV1cG7Gf7Y67mQJHKLXNv7pZ5M2WvzFPpQyVgMB9znNEFMefiFvgn7r72wMott5xBJksEZUfWX5',
  'privkey': 'cUMT7ptJgxrwSm9s5zSUCKUhSNVq1UC9v31vgoSusyukAciNMdDt',
  'pubkey': '02dcb7a11c7172966e58679c0cb207bfb95d119cc55f9fb318d0a522aaff5b9577',
  'pubkeyhash': '3b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb8',
  'index': 0},
 {'path': "m/44'/1'/0'/0/1",
  'address': 'mmUQuAvBYqVpKzayhaYvnJe9VuwkRdSWPu',
  'xprv': 'tprv8khhNkzGYHzyj6e6K8iKuPTCw6YLNRA3tRKCphDT4Yis5AKoPbwFLH7zqay3WqqcsnHCwJZedosPPAXcrHehSKV7MBy57KVPQxyQ3tHyq4g',
  'xpub': 'tpubDHPjXB2WgfgecZftCnNvJo7KW84GXkLxTiuz7DFkUpXFueaa1zkqWmjs1gg5EbtryQSJ81NQyoDwvXGWvis4AtWXeZXh9juHNWhUCGZwfRJ',
  'privkey': 'cVj1Q51vqgtBEYokpcb6RtjMwVbAsaCmrQRyjMTHFm5rGoziYe44',
  'pubkey': '024fca815b769b3e6920eb31081de336

In [3]:
coins['eth']

[{'path': "m/44'/60'/0'/0/0",
  'address': '0x02194a55DDA1B029F70220D53f099F38c629ab36',
  'xprv': 'xprvA2UAbkzcn2ntbeUWWeryfF3uDhgNm4dauGWnvnDix6CsvCzQuvLXjSRM7JTtTTWK9sX95nFVJwYhVGxXqPToYqhz27ChLeVmsCQQXUKHxdM',
  'xpub': 'xpub6FTX1GXWcQMBp8YycgPz2NzdmjWsAXMSGVSPjAdLWRjro1KZTTenHEjpxbBjGGK7wisPZe4htBommydMu8aPzi3Y94xHEdcN3ANiCP23RiY',
  'privkey': '0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc',
  'pubkey': '03b21e2e1bc0b74c298d113c5c305c80cc76e67c4ff29e9b31028ea17328f8bba8',
  'pubkeyhash': 'a85e006b705f905c3e0e82485a33bf5bae8da52e',
  'index': 0},
 {'path': "m/44'/60'/0'/0/1",
  'address': '0x3dFb64A5AdAe2aDece2E252FE5E889d61Ef75122',
  'xprv': 'xprvA2UAbkzcn2ntezNrVAWRNF7yPRSuWDGEu7CdrwoNhYEjuJdeqhFC9zpZ9Uomp4Tuzf47cQ661EzPLErPP9nSvAHoboLqYTTpNd1xfeFGuUP',
  'xpub': 'xpub6FTX1GXWcQMBsUTKbC3RjP4hwTHPufz6GL8EfLCzFsmin6xoPEZSho92znBTHFATi3RFHmhJLpwwR7P9KXcN9f4jAs4YUHRkkyg7spW5kZe',
  'privkey': '0x81b180541f1ae73a48976095a58a342611fcedac9c00758510c0f3c0c63e5710',

In [7]:
priv_key_to_account('BTCTEST', 'cVj1Q51vqgtBEYokpcb6RtjMwVbAsaCmrQRyjMTHFm5rGoziYe44')

In [8]:
create_tx('BTCTEST', 'mmUQuAvBYqVpKzayhaYvnJe9VuwkRdSWPu', 'tb1ql7w62elx9ucw4pj5lgw4l028hmuw80sndtntxt', 0.0001)

In [9]:
send_tx('BTCTEST', 'mmUQuAvBYqVpKzayhaYvnJe9VuwkRdSWPu', 'tb1ql7w62elx9ucw4pj5lgw4l028hmuw80sndtntxt', 0.0001)